## Load Exported R Data

In [2]:
import sys
import pandas as pd
import numpy as np
from typing import Dict, Any

# Add Robyn to path
sys.path.append("/Users/yijuilee/robynpy_release_reviews/Robyn/python/src")

# Import necessary Robyn classes
from robyn.data.entities.mmmdata import MMMData
from robyn.modeling.entities.modeloutputs import ModelOutputs
from robyn.data.entities.hyperparameters import Hyperparameters
from robyn.modeling.pareto.pareto_optimizer import ParetoResult
from utils.data_mapper import load_data_from_json, import_input_collect, import_output_collect, import_output_models

## Export Json data on R notebooks

In [ ]:
# import json

# # Specify the paths where you want to save the JSON files
# input_collect_path = (
#     "/Users/yijuilee/project_robyn/original/Robyn_original_2/Robyn/robyn_api/data/Allocator_InputCollect.json"
# )
# output_models_path = (
#     "/Users/yijuilee/project_robyn/original/Robyn_original_2/Robyn/robyn_api/data/Allocator_OutputModels.json"
# )
# output_collect_path = (
#     "/Users/yijuilee/project_robyn/original/Robyn_original_2/Robyn/robyn_api/data/Allocator_OutputCollect.json"
# )
# # Save each component to a separate JSON file
# with open(input_collect_path, "w") as f:
#     json.dump(InputCollect, f)
# with open(output_models_path, "w") as f:
#     json.dump(OutputModels, f)
# with open(output_collect_path, "w") as f:
#     json.dump(OutputCollect, f)
# print(f"InputCollect exported successfully to {input_collect_path}")
# print(f"OutputModels exported successfully to {output_models_path}")
# print(f"OutputCollect exported successfully to {output_collect_path}")

## Load json data.
* InputCollect
* OutputModels
* outputsArgs

In [3]:
# Load data from JSON exported from R
raw_input_collect = load_data_from_json(
    "/Users/yijuilee/robynpy_release_reviews/Robyn/python/src/tutorials/data/Allocator_InputCollect.json"
)
raw_output_collect = load_data_from_json(
    "/Users/yijuilee/robynpy_release_reviews/Robyn/python/src/tutorials/data/Allocator_OutputCollect.json"
)
raw_output_models = load_data_from_json(
    "/Users/yijuilee/robynpy_release_reviews/Robyn/python/src/tutorials/data/Allocator_OutputModels.json"
)

# Convert R data to Python objects
r_input_collect = import_input_collect(raw_input_collect)
r_output_collect = import_output_collect(raw_output_collect)
python_model_outputs = import_output_models(raw_output_models)

## InputCollect

In [4]:
# Extract individual components
mmm_data = r_input_collect["mmm_data"]
featurized_mmm_data = r_input_collect["featurized_mmm_data"]
holidays_data = r_input_collect["holidays_data"]
hyperparameters = r_input_collect["hyperparameters"]

## Output Collect

In [5]:
pareto_results = r_output_collect["pareto_result"]

cluster_data = r_output_collect["cluster_data"]

## ModelOutputs

In [6]:
model_outputs = python_model_outputs

In [7]:
print(cluster_data)

ClusteredResult(cluster_data=       solID       mape  decomp.rssd     nrmse  nrmse_test  nrmse_train  \
0     1_65_6  70.892934     0.536179  0.249837    0.288892     0.232905   
1    1_150_3  65.780731     0.531938  0.246141    0.296267     0.233498   
2    1_156_9  62.637132     0.531946  0.246173    0.296364     0.233504   
3    1_160_3  65.834692     0.545255  0.246686    0.298407     0.233602   
4    1_166_9  70.679096     0.540862  0.246088    0.296570     0.233461   
..       ...        ...          ...       ...         ...          ...   
97   5_211_9  68.648979     0.530409  0.252300    0.297076     0.229273   
98   5_220_8  69.144987     0.522369  0.264822    0.302831     0.226063   
99   5_221_5  67.902686     0.564497  0.244178    0.296425     0.234521   
100  5_221_9  68.638174     0.533088  0.252435    0.297507     0.229256   
101  5_222_1  68.807062     0.656743  0.243687    0.296002     0.233511   

     nrmse_val  facebook_S_thetas  newsletter_thetas  ooh_S_thetas  ..

In [8]:
print(cluster_data.cluster_ci)

ClusterConfidenceIntervals(cluster_ci=            rn  cluster   n  boot_mean   boot_se        boot_ci    ci_low  \
0   facebook_S        1  10   2.366575  0.155881   [1.26, 3.49]  1.262121   
1   facebook_S        2  16   1.347690  0.107524   [0.43, 2.26]  0.427777   
2   facebook_S        3  20   1.430154  0.077764    [0.7, 2.16]  0.701295   
3   facebook_S        4  17   1.239554  0.151472      [0, 2.57]  0.000000   
4   facebook_S        5  10   4.743634  0.501459   [1.14, 8.31]  1.139199   
5   facebook_S        6   7   2.867470  0.646616      [0, 7.06]  0.000000   
6   facebook_S        7  22   1.424913  0.053152   [0.91, 1.94]  0.905583   
7        ooh_S        1  10   0.084633  0.007558   [0.03, 0.14]  0.030467   
8        ooh_S        2  16   0.085428  0.006194   [0.03, 0.14]  0.032639   
9        ooh_S        3  20   0.100757  0.003836   [0.06, 0.14]  0.064671   
10       ooh_S        4  17   0.287409  0.098495      [0, 1.15]  0.000000   
11       ooh_S        5  10   0.510583

In [9]:
display(cluster_data.n_clusters)

7

In [10]:
display(cluster_data.clusters_pca)

PCAResults(pca_explained=0     25.9507
1     10.6709
2      9.9902
3      8.8544
4      7.8629
5      6.0331
6      5.6946
7      4.5160
8      4.1161
9      3.1435
10     3.0408
11     2.6852
12     1.7013
13     1.6146
14     1.3069
15     1.1680
16     0.8869
17     0.7639
dtype: float64, pcadf=         mape  decomp.rssd     nrmse  nrmse_test  nrmse_train  nrmse_val  \
0    1.000000     0.513412  0.485942    0.386635     0.987566   0.485942   
1    0.927053     0.500238  0.473279    0.398274     0.990857   0.473279   
2    0.882196     0.500263  0.473390    0.398428     0.990890   0.473390   
3    0.927823     0.541604  0.475145    0.401651     0.991431   0.475145   
4    0.996949     0.527959  0.473097    0.398753     0.990649   0.473097   
..        ...          ...       ...         ...          ...        ...   
97   0.967980     0.495487  0.494383    0.399551     0.967412   0.494383   
98   0.975058     0.470514  0.537284    0.408634     0.949605   0.537284   
99   0.957331    

In [11]:
display(cluster_data.clusters_tsne)

None